In [1]:
!pip install spacy
!python -m spacy download en_core_web_md
!pip3 install "paddleocr>=2.6.0.3"
!pip install paddlepaddle==3.0.0b1
!pip3 install paddleclas>=2.4.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
!git clone https://github.com/PaddlePaddle/PaddleOCR.git #cloning the repository of model used for text recognition
!pip install premailer

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 51219, done.
remote: Counting objects: 100% (1537/1537), done.
remote: Compressing objects: 100% (847/847), done.
remote: Total 51219 (delta 804), reused 1314 (delta 676), pack-reused 49682 (from 1)
Receiving objects: 100% (51219/51219), 385.23 MiB | 33.38 MiB/s, done.
Resolving deltas: 100% (35949/35949), done.
Updating files: 100% (2390/2390), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 12.5 MB/s eta 0:00:00


In [3]:
import spacy

In [4]:
nlp = spacy.load("en_core_web_md")

In [6]:
import spacy
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

from paddleocr import PaddleOCR,draw_ocr
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and loads model into memory

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:17<00:00, 231kiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:11<00:00, 908kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:15<00:00, 140kiB/s]

[2024/09/08 01:39:51] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [7]:
df = pd.read_csv('/content/new_wine_reviews.csv')

In [8]:
def POCR(img_path):
  result = ocr.ocr(img_path, cls=True) # model detects text and also the spacial coordinates of text box
  coords={} # saves the coordinates of center of text box and text contained inside box
  heights_sum = 0 # used to calculate average height of text boxes
  n = 0 # counts number of text boxes
  for idx in range(len(result)):
      res = result[idx]
      for line in res:
        x = (line[0][0][0] + line[0][1][0] + line[0][2][0] + line[0][3][0])/4 # x coordinate of center of text box
        y = (line[0][0][1] + line[0][1][1] + line[0][2][1] + line[0][3][1])/4 # y coordinate of center of text box
        heights_sum += abs(line[0][2][1] - line[0][0][1])
        n += 1
        coords[y] = [x, line[1][0]]
  height_avg = heights_sum / n

  # Sort coordinates by keys
  myKeys = list(coords.keys())
  myKeys.sort()
  sorted_coords = {i: coords[i] for i in myKeys}

  out_str = '' # output string
  # Iterate through sorted coordinates
  # To make a string containing text boxes in sequentially as their coordinates
  i = 0
  while i < len(myKeys):
      j = i
        # Collect words while the difference between y coordinates of two boxes is less than height_avg
      while j < len(myKeys) and abs(myKeys[j] - myKeys[i]) < height_avg:
          out_str += sorted_coords[myKeys[j]][1] + ' '
          j += 1
      # Move to the next segment
      i = j
  return out_str

In [14]:
query = POCR('/content/heineken.jpg')

[2024/09/08 01:41:20] ppocr DEBUG: dt_boxes num : 7, elapsed : 0.12426471710205078
[2024/09/08 01:41:20] ppocr DEBUG: cls num  : 7, elapsed : 0.021425247192382812
[2024/09/08 01:41:20] ppocr DEBUG: rec_res num  : 7, elapsed : 0.18193340301513672


In [15]:
brand = df["brand"]
brand = list(brand)

In [27]:
query = query.lower()

In [19]:
sentence_embeddings = [nlp(str(sentence)).vector for sentence in brand]

In [28]:
sentence_embeddings = np.array(sentence_embeddings)
k = 2  # Number of nearest neighbors to find
nn_model = NearestNeighbors(n_neighbors=k, metric='cosine')
nn_model.fit(sentence_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=2)

In [29]:
query_embedding = nlp(query).vector.reshape(1, -1)  # Reshape for compatibility with sklearn
distances, indices = nn_model.kneighbors(query_embedding)

In [30]:
for i in indices:
  print(i)

[172 173]


In [31]:
indices

array([[172, 173]])

In [32]:
for i in indices[0]:
  print("Brand:" + str(df["brand"][i]))
  print("Rating:" + str(df["reviews.rating"][i]))
  print("Title:" + df["reviews.title"][i])
  print("Review:" + df["reviews.text"][i])
  print("-"*50)

Brand:Heineken
Rating:5.0
Title:Good Deal
Review:I like a good price-quality deal for Heineken Lager. My fav easy-going beer. I prefer bottled on and I like to drink it from glasses.
--------------------------------------------------
Brand:Heineken
Rating:5.0
Title:Fresh and cheap
Review:It was very fresh and cheaper than anywhere in the area.
--------------------------------------------------
